## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-28-10-53-13 +0000,nyt,Is the EU Trade Deal With Trump Good for Europe?,https://www.nytimes.com/2025/07/28/world/europ...
1,2025-07-28-10-48-56 +0000,nyt,Trump Administration Live Updates: President t...,https://www.nytimes.com/live/2025/07/28/us/tru...
2,2025-07-28-10-41-55 +0000,bbc,The winners and losers in US-EU trade deal,https://www.bbc.com/news/articles/c14g8gk8vdlo
3,2025-07-28-10-38-18 +0000,nyt,‘Revenge Is Not a Policy’: Israelis Voice Diss...,https://www.nytimes.com/2025/07/28/world/middl...
4,2025-07-28-10-38-00 +0000,wsj,"Thailand, Cambodia Agree to Cease-Fire",https://www.wsj.com/world/asia/thailand-cambod...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2342/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,28
1,trade,15
2,deal,13
24,gaza,11
367,dies,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
156,2025-07-27-17-37-25 +0000,nypost,Trump strikes ‘biggest deal ever made’ with EU...,https://nypost.com/2025/07/27/us-news/trump-eu...,77
6,2025-07-28-10-31-00 +0000,wsj,Stock Market Today: Dow Futures Rise After Tru...,https://www.wsj.com/livecoverage/stock-market-...,71
0,2025-07-28-10-53-13 +0000,nyt,Is the EU Trade Deal With Trump Good for Europe?,https://www.nytimes.com/2025/07/28/world/europ...,71
186,2025-07-27-12-10-56 +0000,nypost,Trump and Ursula von der Leyen to meet for hig...,https://nypost.com/2025/07/27/world-news/trump...,67
43,2025-07-28-08-06-45 +0000,nyt,"Europe, Worried by Other Global Crises, Agrees...",https://www.nytimes.com/2025/07/28/world/europ...,67


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
156,77,2025-07-27-17-37-25 +0000,nypost,Trump strikes ‘biggest deal ever made’ with EU...,https://nypost.com/2025/07/27/us-news/trump-eu...
5,52,2025-07-28-10-35-00 +0000,wsj,Business leaders on both sides of the Atlantic...,https://www.wsj.com/economy/trade/companies-we...
87,36,2025-07-28-00-23-00 +0000,nypost,Thousands of New Yorkers — including ‘Orange i...,https://nypost.com/2025/07/27/us-news/thousand...
101,33,2025-07-27-23-05-59 +0000,nypost,Viral images of starving Gaza boy don’t tell t...,https://nypost.com/2025/07/27/world-news/viral...
110,29,2025-07-27-22-28-40 +0000,nypost,AOC ripped as her NYC district is overrun by s...,https://nypost.com/2025/07/27/us-news/aoc-ripp...
112,29,2025-07-27-22-21-38 +0000,bbc,American musical satirist Tom Lehrer dies at 9...,https://www.bbc.com/news/articles/cpv02yd2714o
187,28,2025-07-27-12-09-57 +0000,bbc,Thailand and Cambodia agree to talks in Malays...,https://www.bbc.com/news/articles/cy854585r32o
122,26,2025-07-27-21-24-50 +0000,nyt,Suspect in Traverse City Walmart Stabbing Atta...,https://www.nytimes.com/2025/07/27/us/walmart-...
20,24,2025-07-28-09-36-19 +0000,nypost,Florida county school board chair deletes comm...,https://nypost.com/2025/07/28/us-news/florida-...
126,24,2025-07-27-20-58-27 +0000,nyt,3 Dead After Small Plane Crashes Off Californi...,https://www.nytimes.com/2025/07/27/us/plane-cr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
